In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.python.keras.utils.vis_utils import plot_model

import sys

py_version = sys.version.split(' ')[0]

print(f'TensorFlow: {tf.__version__}')
print(f'NumPy: {np.__version__}')
print(f'Python: {py_version}')

In [ ]:
tf.constant([[1., 2., 3.], [4., 5., 6.]]) # Matrix

In [ ]:
tf.constant(42) # Scalar

In [ ]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t[:, 1:]

In [ ]:
t[..., 1, tf.newaxis]

In [ ]:
t + 10

In [ ]:
tf.square(t)

In [ ]:
t @ tf.transpose(t)

In [ ]:
K = keras.backend
K.square(K.transpose(t)) + 10

In [ ]:
np.__version__

In [ ]:
# TensorFlow <> NumPy

a = np.array([2., 4., 5.])
tf.constant(a)
t.numpy()
tf.square(a)
np.square(t)

In [ ]:
tf.constant(2., dtype=tf.float64) + tf.constant(40., dtype=tf.float64)

In [ ]:
t2 = tf.constant(40., dtype=tf.float64)
tf.constant(2.0) + tf.cast(t2, tf.float32)

In [ ]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

In [ ]:
v.assign(2 * v)
v[0, 1].assign(42)
v[:, 2].assign([0., 1.])
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100., 200.])

In [ ]:
def huber_fn(y_true, y_pred):
  error = y_true - y_pred
  is_small_error = tf.abs(error) < 1
  squared_loss = tf.square(error) / 2
  linear_loss = tf.abs(error) - 0.5
  return tf.where(is_small_error, squared_loss, linear_loss)

In [ ]:
def create_huber(threshold=1.0):
  def huber_fn(y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < threshold
    squared_loss = tf.square(error) / 2
    linear_loss = threshold * tf.abs(error) - threshold**2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  return huber_fn


In [ ]:
class HuberLoss(keras.losses.Loss):
  def __init__(self, threshold=1.0, **kwargs):
    self.threshold = threshold
    super().__init__(**kwargs)
  def call(self, y_true, y_pred):
    error = y_true - y_pred
    is_small_error = tf.abs(error) < self.threshold
    squared_loss = tf.square(error) / 2
    linear_loss = self.threshold * tf.abs(error) - self.threshold**2 / 2
    return tf.where(is_small_error, squared_loss, linear_loss)
  def get_config(self):
    base_config = super().get_config()
    return {**base_config, "threshold": self.threshold}

In [ ]:
def f(w1, w2):
  return 3 * w1 ** 2 + 2 * w1 * w2

w1, w2 = 5, 3
eps = 1e-6
(f(w1 + eps, w2) - f(w1, w2)) / eps

In [ ]:
(f(w1, w2 + eps) - f(w1, w2)) / eps

In [ ]:
w1, w2 = tf.Variable(5.), tf.Variable(3.)
with tf.GradientTape() as tape:
  z = f(w1, w2)

gradients = tape.gradient(z, [w1, w2])
gradients

In [ ]:
c1, c2 = tf.constant(5.), tf.constant(3.)
with tf.GradientTape() as tape:
  z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
gradients # Returns [None, None]

In [ ]:
with tf.GradientTape() as tape:
  tape.watch(c1)
  tape.watch(c2)
  z = f(c1, c2)

gradients = tape.gradient(z, [c1, c2])
gradients # Returns [tensor 36., tensor 10.]

In [ ]:
def cube(x):
  return x ** 3

cube(2)
cube(tf.constant(2))
cube(tf.constant(2.))

In [ ]:
tf_cube = tf.function(cube)
tf_cube(2)
tf_cube(tf.constant(2.0))

In [ ]:
@tf.function
def tf_cube(x):
  return x ** 3

tf_cube.python_function(2.)

In [ ]:
# Sequential vs Functional

def build_model_with_sequential():
  seq_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
  ])
  return seq_model

def build_model_with_functional():
  input_layer = tf.keras.Input(shape=(28, 28))

  flatten_layer = tf.keras.layers.Flatten()(input_layer)
  first_dense = tf.keras.layers.Dense(128, activation=tf.nn.relu)(flatten_layer)
  output_layer = tf.keras.layers.Dense(10, activation=tf.nn.softmax)(first_dense)

  func_model = Model(inputs=input_layer, outputs=output_layer)

  return func_model

# Build
model = build_model_with_functional()
# model = build_model_with_sequential()

plot_model(model, show_shapes=True, show_layer_names=True)

# Training
mnist = tf.keras.datasets.fashion_mnist
(training_images, training_labels), (test_images, test_labels) = mnist.load_data()
training_images = training_images / 255.0
test_images = test_images / 255.0

model.compile(
  optimizer=tf.optimizers.Adam(),
  loss='sparse_categorical_crossentropy',
  metrics=['accuracy']
)

model.fit(training_images, training_labels, epochs=5)
model.evaluate(test_images, test_labels)